In [1]:
from kafka import KafkaProducer,KafkaConsumer
import threading, logging, time

producerCheck = KafkaProducer(bootstrap_servers='smartaqnet-0-node1:9092',value_serializer=lambda v: json.dumps(v).encode('utf-8'), compression_type='gzip')

consumer = KafkaConsumer("beats", bootstrap_servers='smartaqnet-0-node1:9092')

producerInit = KafkaProducer(bootstrap_servers='smartaqnet-dev.teco.edu:9092',value_serializer=lambda v: json.dumps(v).encode('utf-8'), compression_type='gzip')
producerCheck = KafkaProducer(bootstrap_servers='swarm-node00.teco.edu:9092',value_serializer=lambda v: json.dumps(v).encode('utf-8'), compression_type='gzip')

consumer = KafkaConsumer("beats", bootstrap_servers='smartaqnet-dev.teco.edu:9092')

In [11]:
class Producer(threading.Thread):
    def __init__(self):
        threading.Thread.__init__(self)
        self.stop_event = threading.Event()
        
    def stop(self):
        self.stop_event.set()

    def run(self):
        producer = KafkaProducer(bootstrap_servers='smartaqnet-0-node1:9092')
        t = 0
        while not self.stop_event.is_set():
            t += 1
            message = t
            producer.send('beats', str(message))
            producer.send('beats2', str(message))
            print("Init message to node1:" + str(message))
            time.sleep(1)

        producer.close()

class ConsumerProducer(multiprocessing.Process):
    def __init__(self):
        multiprocessing.Process.__init__(self)
        self.stop_event = multiprocessing.Event()
        
    def stop(self):
        self.stop_event.set()
        
    def run(self):
        consumer = KafkaConsumer(bootstrap_servers= 'smartaqnet-0-node1:9092',
                                 auto_offset_reset='latest',
                                 consumer_timeout_ms=1000)
        
        producer = KafkaProducer(bootstrap_servers='smartaqnet-0-node2:9092')
        consumer.subscribe(['beats'])
        while not self.stop_event.is_set():
            for message in consumer:
                print("Receive message from node1:" + str(message.value))
                producer.send(topic = 'beats2', value = message.value,
                             key = message.key, partition = message.partition,
                             timestamp_ms = message.timestamp)
                print("Copy message to node2:" + message.value)
                if self.stop_event.is_set():
                    break

        consumer.close()
        
class Consumer(multiprocessing.Process):
    def __init__(self):
        multiprocessing.Process.__init__(self)
        self.stop_event = multiprocessing.Event()
        
    def stop(self):
        self.stop_event.set()
        
    def run(self):
        consumer = KafkaConsumer(bootstrap_servers= 'smartaqnet-1-node1:9092',
                                 auto_offset_reset='latest',
                                 consumer_timeout_ms=10000)
        consumer.subscribe(['beats','beats2'])
        
        while not self.stop_event.is_set():
            for message in consumer:
                print("Receive message from backup:" + str(message.value))
                if self.stop_event.is_set():
                    break

        consumer.close()

In [12]:
tasks = [
        Producer(),
        #ConsumerProducer(),
        Consumer()
]

In [13]:
for t in tasks:
        t.start()

Receive message from backup:1
Receive message from backup:1
Init message to node1:1
Receive message from backup:2
Receive message from backup:2
Init message to node1:2
Receive message from backup:3
Receive message from backup:3
Init message to node1:3
Receive message from backup:4
Receive message from backup:4
Init message to node1:4
Receive message from backup:5
Receive message from backup:5
Init message to node1:5
Receive message from backup:6
Receive message from backup:6
Init message to node1:6
Receive message from backup:7
Receive message from backup:7
Init message to node1:7
Receive message from backup:8
Receive message from backup:8
Init message to node1:8
Receive message from backup:9
Receive message from backup:9
Init message to node1:9
Receive message from backup:10
Receive message from backup:10
Init message to node1:10
Receive message from backup:11
Receive message from backup:11
Init message to node1:11
Receive message from backup:12
Receive message from backup:12
Init mes

In [14]:
for t in tasks:
        t.stop()

In [6]:
from kafka import TopicPartition
aconsumer = KafkaConsumer(bootstrap_servers='smartaqnet-1-node1:9092',auto_offset_reset='earlist')

In [53]:
aconsumer.subscribe(["beats","beats2"])

In [61]:
aconsumer.subscription()

AttributeError: 'NoneType' object has no attribute 'copy'

In [38]:
aconsumer.config

{'api_version': (0, 11, 0),
 'api_version_auto_timeout_ms': 2000,
 'auto_commit_interval_ms': 5000,
 'auto_offset_reset': 'earlist',
 'bootstrap_servers': 'smartaqnet-0-node3:9092',
 'check_crcs': True,
 'client_id': 'kafka-python-1.3.5',
 'connections_max_idle_ms': 540000,
 'consumer_timeout_ms': inf,
 'default_offset_commit_callback': <function kafka.consumer.group.<lambda>>,
 'enable_auto_commit': True,
 'exclude_internal_topics': True,
 'fetch_max_bytes': 52428800,
 'fetch_max_wait_ms': 500,
 'fetch_min_bytes': 1,
 'group_id': None,
 'heartbeat_interval_ms': 3000,
 'key_deserializer': None,
 'max_in_flight_requests_per_connection': 5,
 'max_partition_fetch_bytes': 1048576,
 'max_poll_records': 500,
 'metadata_max_age_ms': 300000,
 'metric_group_prefix': 'consumer',
 'metric_reporters': [],
 'metrics_num_samples': 2,
 'metrics_sample_window_ms': 30000,
 'partition_assignment_strategy': (kafka.coordinator.assignors.range.RangePartitionAssignor,
  kafka.coordinator.assignors.roundrobi

In [7]:
aconsumer.assign([TopicPartition('beats', 0)])

In [8]:
aconsumer.assign([TopicPartition('beats2', 0)])

In [9]:
aconsumer.seek_to_beginning()

In [10]:
for message in aconsumer:
    print("Receive message from node3:" + str(message.value) + " " +  str(message.topic))

Receive message from node3:56 beats2
Receive message from node3:57 beats2
Receive message from node3:58 beats2
Receive message from node3:59 beats2
Receive message from node3:60 beats2
Receive message from node3:61 beats2
Receive message from node3:62 beats2
Receive message from node3:63 beats2
Receive message from node3:64 beats2
Receive message from node3:65 beats2
Receive message from node3:66 beats2
Receive message from node3:67 beats2
Receive message from node3:68 beats2
Receive message from node3:69 beats2
Receive message from node3:70 beats2
Receive message from node3:71 beats2
Receive message from node3:72 beats2
Receive message from node3:73 beats2
Receive message from node3:74 beats2
Receive message from node3:75 beats2
Receive message from node3:76 beats2
Receive message from node3:77 beats2
Receive message from node3:78 beats2
Receive message from node3:79 beats2
Receive message from node3:80 beats2
Receive message from node3:81 beats2
Receive message from node3:82 beats2
R

KeyboardInterrupt: 

In [44]:
aRec.timestamp

'43'

In [49]:
producer = KafkaProducer(bootstrap_servers='smartaqnet-0-node3:9092',value_serializer=lambda v: json.dumps(v).encode('utf-8'), compression_type='gzip')

In [50]:
producer.send('beats', "someThing")

In [28]:
from kafka import KafkaProducer,KafkaConsumer,BrokerConnection
from kafka.client import KafkaClient 

In [29]:
aKafkaClient = KafkaClient(bootstrap_servers = 'smartaqnet-0-node3:9092')

In [35]:
aKafkaClient.connected(1003)

True

In [39]:
aKafkaClient.set_topics("aTopic")

In [45]:
aKafkaClient.check_version()

(0, 11, 0)

In [42]:
aBrocker = BrokerConnection("smartaqnet-0-node1","9092",0,api_version=(0,10))

In [26]:
aBrocker.connect()

'<connecting>'

In [43]:
aBrocker.connected()

False

In [44]:
aBrocker.connecting()

False

In [41]:
aBrocker.state

'<connecting>'

In [46]:
producer = KafkaProducer(bootstrap_servers='smartaqnet-0-node3:9092',value_serializer=lambda v: json.dumps(v).encode('utf-8'), compression_type='gzip')

In [47]:
for i in range(1000):
    producer.send('bTopic', "someThing")

In [48]:
#Consumer
from kafka import TopicPartition
aconsumer = KafkaConsumer(bootstrap_servers='smartaqnet-0-node3:9092',auto_offset_reset='earlist',consumer_timeout_ms=10000,group_id="agroup")
#aconsumer.subscribe(["aTopic"])
aconsumer.assign([TopicPartition('beatsGlobal', 0)])
aconsumer.seek_to_beginning()

In [12]:
for message in aconsumer:
    print("Receive message from node1:" + str(message.value) + " " +  str(message.topic))

In [53]:
aconsumer.assignment()

{TopicPartition(topic='beatsGlobal', partition=0)}

In [27]:
producer.send(topic='aTopic', value="someThing")